<a href="https://colab.research.google.com/github/davidislip/RedBlueSetCovering/blob/main/Results_Stochastic_RBSC_Scenarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stochastic Red Blue Set Covering 

This notebook focuses on the scenario formulation of the stochastic RBSC problem. The advantage of the scenario formulation is that there are no assumptions regarding the particular structure of the uncertainty. Specifically, the scenario approach allows for detailed modelling of the random color dependence structure between elements. 

Let $\xi \in \Xi$ represent a scenario out of the possible scenarios. 

In [ ]:
!sudo python -m pip install gurobipy==9.1.2
!pip install netgraph
import gurobipy as gp
import pandas as pd
import numpy as np
from gurobipy import GRB
from google.colab import drive 
from itertools import product
import math, sys, time
from netgraph import Graph, InteractiveGraph, EditableGraph
import matplotlib.pyplot as plt
import multiprocessing
import networkx as nx
import random
import pickle as pkl

drive.mount('/content/gdrive')
pth = 'gdrive/My Drive/Colab Notebooks/'
sys.path.append(pth + 'RBSC/')
%matplotlib inline

     |████████████████████████████████| 11.1 MB 5.5 MB/s 
     |████████████████████████████████| 77 kB 339 kB/s 
     |████████████████████████████████| 246 kB 12.1 MB/s 
     |████████████████████████████████| 41 kB 207 kB/s 
  Created wheel for netgraph: filename=netgraph-4.2.5-py3-none-any.whl size=84060 sha256=3ca72463df99d66fb8d2fb144ef2b67a13d03143c68846e428964686fe30da37
  Stored in directory: /root/.cache/pip/wheels/e6/b6/d6/5eecbfe16ae7463d8c9cb589b50f33d5d4c10a80e7d6aa22f0
Successfully built netgraph
Mounted at /content/gdrive


In [ ]:
#read in licence info
gurobi_licence = pd.read_csv(pth +'RBSC/gurobi.csv')
print("Required info for Gurobi:", gurobi_licence.columns)
try:
  #web license try to access it via uoft
  e = gp.Env(empty=True)
  #e.setParam('OutputFlag', 0)
  e.setParam('WLSACCESSID', gurobi_licence.WLSACCESSID[0])
  e.setParam('LICENSEID', gurobi_licence.LICENSEID[0])
  e.setParam('WLSSECRET', gurobi_licence.WLSSECRET[0])
  e.start()
except: 
  !chmod 755 /content/gdrive/My\ Drive/Colab\ Notebooks/SVM\ MVO/gurobi/grbgetkey
  !/content/gdrive/My\ Drive/Colab\ Notebooks/SVM\ MVO/gurobi/grbgetkey {gurobi_licence.LOCAL[0]}
  e = gp.Env(empty=True)
  #chmod 755 grbgetkey
  e.start()

Required info for Gurobi: Index(['WLSACCESSID', 'LICENSEID', 'WLSSECRET', 'LOCAL'], dtype='object')
Changed value of parameter WLSACCESSID
Changed value of parameter LICENSEID
Changed value of parameter WLSSECRET
info  : grbgetkey version 9.1.2, build v9.1.1rc0-30-g8af970cb
info  : Contacting Gurobi license server...
info  : License file for license ID 782118 was successfully retrieved
info  : License expires at the end of the day on 2022-05-19
info  : Saving license file...

In which directory would you like to store the Gurobi license file?
[hit Enter to store it in /opt/gurobi]: 

info  : License 782118 written to file /opt/gurobi/gurobi.lic
Academic license - for non-commercial use only - expires 2022-05-19
Using license file /opt/gurobi/gurobi.lic


In [ ]:
from rbsc import *

Changed value of parameter WLSACCESSID
Changed value of parameter LICENSEID
Changed value of parameter WLSSECRET
Academic license - for non-commercial use only - expires 2022-05-19
Using license file /opt/gurobi/gurobi.lic


In [ ]:

def defineinstance(N,n_elem, Scenarios, maxBlueProb, TransmissionProb, CoverFactor, N_nodes, lambd, Plot = False):
  Sets = {}
  n=np.arange(n_elem)+1
  # Adapated from https://stackoverflow.com/questions/71024509/create-different-disconnected-graphs-from-a-set-of-fixed-nodes-in-networkx
  #randomly permuting nodes
  Elements=np.random.permutation(n)

  N_graphs= round(math.sqrt(n_elem))
  #assign the random modes to each graph
  random_graphs_nodes=[Elements[N_nodes*i:N_nodes*(i+1)] for i in range(N_graphs)]

  #create random graphs
  r_g=[nx.erdos_renyi_graph(n=N_nodes,p=0.5) for _ in range(N_graphs)]

  #relabel the nodes in each graph according to random_graphs_nodes
  mappings=[]
  for i in range(N_graphs):
    mappings.append({j:random_graphs_nodes[i][j] for j in range(N_nodes)})
    r_g[i]=nx.relabel_nodes(r_g[i], mappings[i]) 

  if Plot:
    #plot result
    fig=plt.figure(figsize=(15,6))
    for i in range(N_graphs):
      plt.subplot(1,N_graphs,i+1)
      plt.xlabel('Graph '+str(i+1))
      plt.tight_layout()
      # nx.draw(r_g[i],pos=pos,with_labels=True,node_color=colors[i])
      g = Graph(r_g[i],node_labels=True, node_layout = 'spring',
          node_label_fontdict=dict(size=10), node_label_offset=0.05, node_size=3, edge_width=0.4)

  NodeProbability = {e:np.random.random()*maxBlueProb for e in Elements}
  RedScenarios = {}
  BlueScenarios = {}
  for xi in range(Scenarios):
    Blues_xi = {}
    Reds_xi = {}
    for e in Elements:
      if np.random.random() <= NodeProbability[e]:
        Blues_xi[e] = 'B'
    if Blues_xi == {}:
      Blues_xi[e] = 'B'
    InitialBlues= list(Blues_xi.keys())
    for b in InitialBlues:
      for i in range(N_graphs):
        if b in r_g[i].nodes():
          #propagate the fraud
          infectedByb = []
          propagate(r_g[i],b,infectedByb, TransmissionProb)
          for infected in infectedByb:
            Blues_xi[infected] = 'B'
    Reds_xi = {e:'R' for e in Elements if e not in Blues_xi.keys()}
    BlueScenarios[xi] = Blues_xi
    RedScenarios[xi] = Reds_xi

    for xi in BlueScenarios.keys():#test 
      if set(BlueScenarios[xi]).union(set(RedScenarios[xi])) != set(Elements):
        print("Something is wrong")

  #form the extensive problem 
  ExtensiveBlues = {(b, xi):'B' for xi in BlueScenarios.keys() for b in BlueScenarios[xi].keys()}
  #blues are all combinations of xi and blue elements 
  #reds are all combinations of xi and red elements + one red for each x, blue pair with weight lambda 
  #sets are the given sets plus the super sets for each xi, blue element pair
  ExtensiveReds = {(r, xi): 'R' for xi in RedScenarios.keys() for r in RedScenarios[xi].keys()}
  ExtensiveRedsWeights = {(r, xi): 1/Scenarios for xi in RedScenarios.keys() for r in RedScenarios[xi].keys()}

  BlueMapping = {}
  LastElement = max(Elements)
  k = 1
  for (b, xi) in ExtensiveBlues.keys():
    BlueMapping[(b, xi)] = (LastElement + k, xi)
    ExtensiveReds[(LastElement + k, xi)] = 'Super'
    ExtensiveRedsWeights[(LastElement + k, xi)] = lambd/Scenarios
    k = k + 1

  #form first stage sets that cover elements that are ever blue
  k = 0
  EverBlues = set(b for (b,xi) in ExtensiveBlues.keys())
  UncoveredBlues = [b for b in EverBlues]
  #randomly sample until all the blues are covered 
  while (k < N or UncoveredBlues != []):
    n_k = random.sample(range(1,(1+len(Elements))//CoverFactor),1)[0]
    DoesNotCoverAnyBlue = True
    while DoesNotCoverAnyBlue:
      SetCandidate = random.sample(list(Elements), n_k)
      BlueCoveredBool = [b in SetCandidate for b in EverBlues]
      if True in BlueCoveredBool:
        DoesNotCoverAnyBlue = False
    if UncoveredBlues == []:
      ind = 'Set' + str(k)
      Sets[ind] = SetCandidate
      k = k+1

    CoversNewBlue = False
    for element in EverBlues:
      if element in SetCandidate and element in UncoveredBlues:
        UncoveredBlues.remove(element)
        CoversNewBlue = True
    if CoversNewBlue:
      ind = 'Set' + str(k)
      Sets[ind] = SetCandidate
      k = k+1
  #element scenario pairs 
  ExtensiveFormElements = set(product(Elements, range(Scenarios)))

  ScenarioFormSets = {}

  #appending the base elements for each scenario
  for S in Sets.keys():
    ScenarioFormSets[S] = set()
    for e, xi in ExtensiveFormElements:
      probability_swap = 0.1
      if e in Sets[S] and probability_swap <= np.random.random():
        ScenarioFormSets[S].add((e,xi))
      else:
        RandomSet = random.choice(list(Sets.keys()))
        try:
          ScenarioFormSets[RandomSet].add((e,xi))
        except:
          ScenarioFormSets[RandomSet] = []

  #appending the super sets
  SuperLookup = {}
  bLookup = {}
  k = 0
  for (b, xi) in ExtensiveBlues:
    (r, xi2) = BlueMapping[(b,xi)]
    ScenarioFormSets['Super'+str(k)] = [(b, xi), (r, xi)]
    RedScenarios[xi][r] = 'R' #include the new red elements here
    SuperLookup[(b, xi)] =  'Super'+str(k)
    bLookup[('Super'+str(k), xi)] = b
    k += 1

  SetsIndexedbyScenario = {}
  for xi in range(Scenarios):
    SetsinScenario = {}
    for S in ScenarioFormSets.keys():
      Temp = []
      for pair in ScenarioFormSets[S]:
        if pair[1] == xi: 
          Temp.append(pair[0])
      if Temp != []:
        SetsinScenario[S] = Temp
    SetsIndexedbyScenario[xi] = SetsinScenario
  SetsinScenario

  #getting the weights indexed by scenario
  WeightsIndexedbyScenario = {}
  for xi in range(Scenarios):
    WeightsinScenario = {}
    for (r, xi_2), wgt in ExtensiveRedsWeights.items():
      if xi_2 == xi:
        WeightsinScenario[r] = wgt
    WeightsIndexedbyScenario[xi] = WeightsinScenario
    
  return Sets, ExtensiveReds, ExtensiveBlues, ScenarioFormSets, ExtensiveRedsWeights, SetsIndexedbyScenario, RedScenarios, BlueScenarios, WeightsIndexedbyScenario, SuperLookup, bLookup, BlueMapping

### Peleg Approximation Algorithm 



In [ ]:
# Start foo as a process
def test_peleg(Sets, ExtensiveReds, ExtensiveBlues, ScenarioFormSets, ExtensiveRedsWeights):
  q = multiprocessing.Queue()
  p = multiprocessing.Process(target=LowDeg2, name="Foo", args=(ExtensiveReds, ExtensiveBlues, ScenarioFormSets, ExtensiveRedsWeights, q,))
  start = time.time()
  end = time.time()
  p.start()
  success = False 
  ### get the dummy solution 
  FeasibleSuperSets = set(ScenarioFormSets.keys()) - Sets.keys()
  BestCover = {S:0 for S in ScenarioFormSets.keys()}
  MinWgt = 0
  for S in FeasibleSuperSets:
    BestCover[S] = 1
    MinWgt += NumRedsinS(S, ExtensiveReds, ExtensiveRedsWeights)
  
  while end - start <= LIMIT:
  # Cleanup
    p.join(5)
    if not q.empty():
      [BestCover, MinWgt] = q.get()
      end = time.time()
      print("Terminated Alg with Wgt ", MinWgt)
      success = True 
      break
    # If thread is active
    end = time.time()

  p.terminate()
  return BestCover, MinWgt, end - start, success

### Benders functions 

In [ ]:
def subproblem(xi,SetsIndexedbyScenario, RedScenarios, WeightsIndexedbyScenario, vals_x,  output = False):
  ### dual subproblem implementation 
  m_sub = gp.Model()
  #red element set combinatations where r is in S 
  if output == False:
    m_sub.Params.OutputFlag = 0

  RedSetCombos = {(r,S) for S in SetsIndexedbyScenario[xi].keys() for r in RedScenarios[xi].keys() if r in SetsIndexedbyScenario[xi][S]}
  pi = m_sub.addVars(RedSetCombos, name='pi', lb = 0, vtype = GRB.CONTINUOUS)

  for r in RedScenarios[xi].keys():
    SetsContainingr = {Set for Set in SetsIndexedbyScenario[xi].keys() if r in SetsIndexedbyScenario[xi][Set]}
    if SetsContainingr!= None:
      m_sub.addConstr(gp.quicksum(pi[(r,S)] for S in SetsContainingr) <= WeightsIndexedbyScenario[xi][r])

  m_sub.setObjective(gp.quicksum(vals_x[S]*pi[(r,S)] for (r,S) in pi.keys()), GRB.MAXIMIZE)
  m_sub.optimize()

  vals_pi = {(r,S):pi[(r,S)].x for (r,S) in pi.keys()}
  ObjVal = m_sub.ObjVal
  return vals_pi, ObjVal
# vals_pi, ObjVal = subproblem(xi,SetsIndexedbyScenario, RedScenarios, WeightsIndexedbyScenario, vals_x, output = True)
# ObjVal

In [ ]:
# Callback - use lazy constraints (and user cuts) to add benders cuts
def benders(model, where):
    if where == GRB.Callback.MIPSOL:    # This is the LAZY CONSTRAINT, called only when an INTEGER solution is found
                                        # (mandatory, you can't find the optimal solution w/o the lazy constraints)
      global counter
      global PrintInfo
      if PrintInfo:
        print('**************************************************')
        print('Integer solution is found...')
      
      vals_x_firststage = model.cbGetSolution(x)
      #theta = [var for var in model.getVars() if 'theta' in var.varName]
      #this causes theta[0] to refer to the variable
      vals_theta = model.cbGetSolution(theta)
      UB = 0
      LB = 0

      for xi in range(Scenarios):
        FirstStageSets = set(SetsIndexedbyScenario[xi].keys()).intersection(BendersSets)
        SuperSetsinScenario = set(SetsIndexedbyScenario[xi].keys()).intersection(SuperSets)

        vals_x = {S: 0 for S in SetsIndexedbyScenario[xi].keys()}
        elementscovered = set()
        for S in vals_x_firststage.keys():
          if S in vals_x.keys():
            vals_x[S] = vals_x_firststage[S]
            if vals_x[S] > 0.9:
              elementscovered = elementscovered.union(set(SetsIndexedbyScenario[xi][S]))
        #get uncovered blue elements 
        uncoveredblues = set(BlueScenarios[xi].keys()) - elementscovered
        if uncoveredblues!= set():
          for b in uncoveredblues:
            vals_x[SuperLookup[(b, xi)]] = 1
        # print(vals_x)
        # print(SuperSetsinScenario)
        #full solution including super sets is in vals_x
        vals_pi, SubVal = subproblem(xi,SetsIndexedbyScenario, RedScenarios, WeightsIndexedbyScenario, vals_x)

        RedSetCombos_1 = {(r,S) for S in FirstStageSets 
                          for r in set(SetsIndexedbyScenario[xi][S]).intersection(set(RedScenarios[xi].keys()))}

        RedSetCombos_2 =  {(r,S) for S in SuperSetsinScenario 
                          for r in set(SetsIndexedbyScenario[xi][S]).intersection(set(RedScenarios[xi].keys()))}
        SetsContainingBlues = {}
        for b in BlueScenarios[xi].keys():
          SetsContainingBlues[b] = set()
          for S in FirstStageSets:
            if b in SetsIndexedbyScenario[xi][S]:
              SetsContainingBlues[b].add(S)
        Triples = {(r,Sb,FirstStageS) for (r,Sb) in RedSetCombos_2 for FirstStageS in SetsContainingBlues[bLookup[(Sb, xi)]]}
        # print(xi)
        # print({S for S in vals_x if vals_x[S] == 1})
        # print({(r,S):vals_pi[(r,S)] for (r,S) in RedSetCombos_2})
        # print(SetsIndexedbyScenario[xi])
        # print(vals_x)
        if PrintInfo:
          print("\n\n\n")
          print("At iteration " + str(counter) + ", ")
        
          #print("x = ", list(vals_x))
          print("Theta val = ", vals_theta[xi])
          print("Master objective value (LB) = ", vals_theta[xi])
          #print("pi = ", list(vals_pi.values()))
          print("Benders cut bound = ", SubVal)
        
        status = SubVal - vals_theta[xi] >= 0.00001
        if PrintInfo:
          print("Is their any violation?", status)
        if status:
            if PrintInfo: 
              print('Benders cut added...')
            model.cbLazy(theta[xi] >= 
                    sum(x[S]*vals_pi[(r,S)] for (r,S) in RedSetCombos_1)
                    +sum(1*vals_pi[(r,S)]*vals_x[S] for (r,S) in RedSetCombos_2)
                    - sum(x[FirstStageS]*vals_pi[(r,Sb)]*vals_x[Sb] for (r,Sb,FirstStageS) in Triples))
        else:
            if PrintInfo:
              print('No violation')
        UB += SubVal
        LB += vals_theta[xi]
        if PrintInfo:
          print("UB - LB = ", UB - LB)
          print("\n\n\n")

          print('**************************************************')
    
      counter += 1


### Dual Decomposition Functions

In [ ]:

def subproblemBDDP1(xi,SetsIndexedbyScenario, RedScenarios, BlueScenarios, WeightsIndexedbyScenario, vals_x,  output = False):
  m_sub = gp.Model()
  #red element set combinatations where r is in S 
  if output == False:
    m_sub.Params.OutputFlag = 0
  y = m_sub.addVars(RedScenarios[xi].keys(), obj = WeightsIndexedbyScenario[xi], name='y', lb = 0, vtype = GRB.CONTINUOUS)
  z = m_sub.addVars(SetsIndexedbyScenario[xi].keys(), obj = 0, name='z', lb = 0, vtype = GRB.CONTINUOUS)

  for S in SetsIndexedbyScenario[xi].keys():
    m_sub.addConstrs((y[r] >= z[S] for r in set(SetsIndexedbyScenario[xi][S]).intersection(set(RedScenarios[xi].keys()))) )
  blue_constrs = []
  for b in BlueScenarios[xi].keys():
    c = m_sub.addConstr(gp.quicksum(z[S] for S in \
                            {Set for Set in SetsIndexedbyScenario[xi].keys() if b in SetsIndexedbyScenario[xi][Set]})\
                >= 1)
    blue_constrs.append(c)

  LinkConstraint = m_sub.addConstrs((z[S] == vals_x[S] for S in z.keys()))### does not matter 
  m_sub._LinkConstraint = LinkConstraint #for the call back

  m_sub.optimize()
  # vals_v = None
  # if m_sub.status  == 3:
  #   #infeasible then we have to solve the feasibility problem
  #   v  = m_sub.addVars(BlueScenarios[xi].keys(), name='v', lb = 0, vtype = GRB.CONTINUOUS)
  #   for b in BlueScenarios[xi].keys():
  #     m_sub.remove(blue_constrs)
  #     blue_constrs[b].RHS = 1 - v[b]
  #     m_sub.addConstr(v[b] + gp.quicksum(z[S] for S in \
  #                           {Set for Set in SetsIndexedbyScenario[xi].keys() if b in SetsIndexedbyScenario[xi][Set]})\
  #               >= 1)
  #   m_sub.setObjective(v.sum(), GRB.MINIMIZE)
  #   m_sub.optimize()
  #   vals_v = m_sub.getAttr('x', v)
  vals_y = m_sub.getAttr('x', y)
  vals_z = m_sub.getAttr('x', z)
  Pi = m_sub.getAttr('Pi', m_sub._LinkConstraint)
  BDD1ObjVal = m_sub.ObjVal
  return vals_y, vals_z, Pi, BDD1ObjVal


In [ ]:
#subproblem for phase 2 
def subproblemBDDP2(xi,SetsIndexedbyScenario, RedScenarios, BlueScenarios, WeightsIndexedbyScenario, vals_x, Pi, J = None, output = False):
  m_lagrangian = gp.Model()
  #red element set combinatations where r is in S 
  if output == False:
    m_lagrangian.Params.OutputFlag = 0
  y_lagrangian = m_lagrangian.addVars(RedScenarios[xi].keys(), 
                                      obj = WeightsIndexedbyScenario[xi], 
                                      name='y', lb = 0, vtype = GRB.CONTINUOUS)
  z_lagrangian = m_lagrangian.addVars(SetsIndexedbyScenario[xi].keys(),
                                      obj = 0, name='z', vtype = GRB.BINARY)
  m_lagrangian.setObjective(
      gp.quicksum(WeightsIndexedbyScenario[xi][r]*y_lagrangian[r] for r in y_lagrangian.keys()) 
      - gp.quicksum(z_lagrangian[S]*Pi[S] for S in SetsIndexedbyScenario[xi].keys()) 
      + gp.quicksum(vals_x[S]*Pi[S] for S in SetsIndexedbyScenario[xi].keys()) )
  
  for S in SetsIndexedbyScenario[xi].keys():
    m_lagrangian.addConstrs((y_lagrangian[r] >= z_lagrangian[S] 
                             for r in set(SetsIndexedbyScenario[xi][S]).intersection(set(RedScenarios[xi].keys()))))

  for b in BlueScenarios[xi].keys():
    m_lagrangian.addConstr(gp.quicksum(z_lagrangian[S] for S in \
                            {Set for Set in SetsIndexedbyScenario[xi].keys() if b in SetsIndexedbyScenario[xi][Set]})\
                >= 1)
  if J != None:
    for S in J:
      z_lagrangian[S].vtype = GRB.CONTINUOUS
      z_lagrangian[S].lb = 0
      z_lagrangian[S].ub = 1
  m_lagrangian.optimize()
  vals_y_lagrang = m_lagrangian.getAttr('x', y_lagrangian)
  vals_z_lagrang = m_lagrangian.getAttr('x', z_lagrangian)
  LagrangianObjVal = m_lagrangian.ObjVal
  return vals_y_lagrang, vals_z_lagrang, LagrangianObjVal

#vals_y_lagrang, vals_z_lagrang, LagrangianObjVal = subproblemBDDP2(xi,SetsIndexedbyScenario, RedScenarios, BlueScenarios, WeightsIndexedbyScenario, vals_x, Pi, J = None, output = False)


In [ ]:

# Callback - use lazy constraints (and user cuts) to add benders cuts
def BDD(model, where):
  global counter
  global PrintInfo
  global IntegerX
  global delta
  global epsilon
  global xWarm
  if where == GRB.Callback.MIPSOL:    # This is the LAZY CONSTRAINT, called only when an INTEGER solution is found
                                        # (mandatory, you can't find the optimal solution w/o the lazy constraints)
    if PrintInfo:
      print('**************************************************')
      print('Integer solution is found...')
    #change to warm start here if you'd like
    vals_x_firststage = model.cbGetSolution(x)
    #theta = [var for var in model.getVars() if 'theta' in var.varName]
    #this causes theta[0] to refer to the variable
    vals_theta = model.cbGetSolution(theta)
    UB = 0
    LB = 0
    
    for xi in range(Scenarios):

        FirstStageSets = set(SetsIndexedbyScenario[xi].keys()).intersection(BendersSets)
        SuperSetsinScenario = set(SetsIndexedbyScenario[xi].keys()).intersection(SuperSets)
        #for input into the subproblem
        vals_x = {S: 0 for S in SetsIndexedbyScenario[xi].keys()}
        elementscovered = set()
        for S in vals_x_firststage.keys():
          if S in vals_x.keys():
            vals_x[S] = vals_x_firststage[S]
            if vals_x[S] > 0.9:
              elementscovered = elementscovered.union(set(SetsIndexedbyScenario[xi][S]))
        #get uncovered blue elements 
        uncoveredblues = set(BlueScenarios[xi].keys()) - elementscovered
        if uncoveredblues!= set():
          for b in uncoveredblues:
            vals_x[SuperLookup[(b, xi)]] = 1

                  #add the resulting cut
        SetsContainingBlues = {}
        for b in BlueScenarios[xi].keys():
          SetsContainingBlues[b] = set()
          for S in FirstStageSets:
            if b in SetsIndexedbyScenario[xi][S]:
              SetsContainingBlues[b].add(S)

        # build the subproblem (it's better to build it once, outside the callback, and then modify it inside - left as an exercise)
        vals_y, vals_z, Pi, BDD1ObjVal = subproblemBDDP1(xi,SetsIndexedbyScenario, 
                                                        RedScenarios, BlueScenarios,
                                                        WeightsIndexedbyScenario, 
                                                        vals_x,  output = False)

        if PrintInfo:
          print("\n\n\n")
          print("At iteration " + str(counter) + ", ")
          print("Theta val = ", vals_theta[xi])
          print("Master objective value (LB) = ", vals_theta[xi])
          print("Benders cut bound = ",BDD1ObjVal)
      
        status = BDD1ObjVal - vals_theta[xi] >= 0.001
        if PrintInfo:
          print("Is their any violation?", status)
        if status:
          if PrintInfo: 
            print('Benders cut added...')
            print([vals_z[S]*Pi[S] for S in x.keys()])
            print(vals_z)
            print(vals_x)
            print(Pi)
          # $$\theta \geq 
          # \sum_{r \in \bar{R}(\xi)} c_r \bar{y}_r + 
          # \sum_{S \in \mathcal{S}} \bar{\lambda}_S (x_S - \bar{z}_S) +
          # \sum_{b \in \bigcup_{b\in B(\xi)}}\bar{\lambda}_{S_b} (x^*_{S_b}-\bar{z}_{S_b}) -
          # \sum_{b \in \bigcup_{b\in B(\xi)}} \sum_{S \ni b \subseteq \mathcal{S}}\bar{\lambda}_{S_b} x^*_{S_b} x_S
          # $$ (8)

          BlueElementSetPairs = {(Sb, FirstStageS) for Sb in SuperSetsinScenario for FirstStageS in SetsContainingBlues[bLookup[(Sb, xi)]]}

          model.cbLazy(theta[xi] >= sum(vals_y[r]*WeightsIndexedbyScenario[xi][r] for r in vals_y.keys())
                        + gp.quicksum(x[S]*Pi[S] for S in FirstStageSets)
                        - sum(vals_z[S]*Pi[S] for S in FirstStageSets)
                        + sum(Pi[Sb]*(vals_x[Sb] - vals_z[Sb]) for Sb in SuperSetsinScenario)
                        - gp.quicksum(Pi[Sb]*vals_x[Sb]*x[S] for (Sb, S) in BlueElementSetPairs))
          # else:
          #   model.cbLazy(0 >= vals_v.sum()
          #                 + gp.quicksum(x[S]*Pi[S] for S in FirstStageSets)
          #                 - sum(vals_z[S]*Pi[S] for S in x.keys())
          #                 + sum(Pi[Sb]*(vals_x[Sb] - vals_z[Sb]) for Sb in SuperSetsinScenario)
          #                 - gp.quicksum(Pi[Sb]*vals_x[Sb]*x[S] for (Sb, S) in BlueElementSetPairs))
        else:
          if PrintInfo:
            print('No violation')
        #update bounds
        UB += BDD1ObjVal
        LB += vals_theta[xi]
        if PrintInfo:
          print("UB - LB = ", UB - LB)
          print("\n\n\n")
        #if we are at the root node and have added a cut
        if model.cbGet(GRB.Callback.MIPSOL_NODCNT) < 1 and IntegerX and status:
          #begin phase #2 by solving the inner lagragian

          #first change the variable types for the fractional master problem solutions
          #force them to be integer, let all the already integer variables 
          #be relaxed and free
          J = {S for S in vals_x.keys() if vals_x[S] 
                <= math.ceil(vals_x[S]) - epsilon and
                vals_x[S] >= math.floor(vals_x[S]) + epsilon}

          #solve the inner lagrange problem 
          vals_y_lagrang, vals_z_lagrang, LagrangianObjVal = subproblemBDDP2(xi, 
                                                                             SetsIndexedbyScenario, 
                                                                             RedScenarios, 
                                                                             BlueScenarios, 
                                                                             WeightsIndexedbyScenario, 
                                                                             vals_x, 
                                                                             Pi, J)
          #phase 2 solve 2.6 to obtain a cut, then go to phase 3

          if PrintInfo:
            print('Lagrangian cut added...')
          #add the resulting cut
          model.cbLazy(theta[xi] >= sum(vals_y_lagrang[r]*WeightsIndexedbyScenario[xi][r] for r in vals_y_lagrang.keys())
              + gp.quicksum(x[S]*Pi[S] for S in FirstStageSets)
              - sum(vals_z_lagrang[S]*Pi[S] for S in FirstStageSets)
              + sum(Pi[Sb]*(vals_x[Sb] - vals_z_lagrang[Sb]) for Sb in SuperSetsinScenario)
              - gp.quicksum(Pi[Sb]*vals_x[Sb]*x[S] for (Sb, S) in BlueElementSetPairs))
          
           #project the cut upwards using the heuristic program
          #initialize parameters for loop
          t = 0 
          change = 1
          lambd_last = Pi
          y_last = vals_y_lagrang
          z_last = vals_z_lagrang
          old_eta = LagrangianObjVal
          delta_t = delta/(t+1)

          #set up the lifting problem
          m_lifting.remove(m_lifting._lambd)
          lambd = m_lifting.addVars(SetsIndexedbyScenario[xi].keys(), name='lambda', vtype = GRB.CONTINUOUS)
          m_lifting._lambd = lambd
         
          #white eta is changing, update the lifting program and then 
          #solve it, then add the resulting strengthed cut to the master
          while t <= IterLim and change > 0.5:
            if PrintInfo:
              print("Lifting Iteration: ", t)
            m_lifting.setObjective(eta  - 
                                   (delta_t/2)*gp.quicksum(
                                       (lambd_last[S] - lambd[S])*(lambd_last[S] - lambd[S])
                                       for S in lambd.keys() ), GRB.MAXIMIZE)
            m_lifting.addConstr(
                eta <= sum(y_last[r]*WeightsIndexedbyScenario[xi][r] for r in y_last.keys()) + 
                gp.quicksum((vals_x[S] - z_last[S])*lambd[S] for S in lambd.keys())
                )
            
            m_lifting.optimize()
            delta_t = delta/(t+1)
            lambd_last = m_lifting.getAttr('x', lambd)
            new_eta = eta.x
            change = abs(new_eta - old_eta)
            old_eta = new_eta
            #resolve inner problem with new lambda 
            vals_y_lagrang, vals_z_lagrang, LagrangianObjVal = subproblemBDDP2(xi, 
                                                                              SetsIndexedbyScenario, 
                                                                              RedScenarios, 
                                                                              BlueScenarios, 
                                                                              WeightsIndexedbyScenario, 
                                                                              vals_x, 
                                                                              lambd_last, J)

            #add the resulting cut

            model.cbLazy(theta[xi] >= sum(vals_y_lagrang[r]*WeightsIndexedbyScenario[xi][r] for r in vals_y_lagrang.keys())
              + gp.quicksum(x[S]*lambd_last[S] for S in FirstStageSets)
              - sum(vals_z_lagrang[S]*lambd_last[S] for S in FirstStageSets)
              + sum(lambd_last[Sb]*(vals_x[Sb] - vals_z_lagrang[Sb]) for Sb in SuperSetsinScenario)
              - gp.quicksum(lambd_last[Sb]*vals_x[Sb]*x[S] for (Sb, S) in BlueElementSetPairs))

            if PrintInfo:
              print('Lifted Lagrangian cut added...')
              print('eta ', new_eta)
              print('change in eta ', change)
              print('**************************************************')
            #update 
            y_last = vals_y_lagrang
            z_last = vals_z_lagrang
            t = t+1

          m_lifting.remove(m_lifting.getConstrs())
        counter += 1
  #only at the root node
  if where == GRB.Callback.MIPNODE and model.cbGet(GRB.Callback.MIPNODE_NODCNT) < 1:    # This is the LAZY CONSTRAINT, called only when an INTEGER solution is found
                                    # (mandatory, you can't find the optimal solution w/o the lazy constraints)
    if PrintInfo:
      print('**************************************************')
      print('Node Relaxation...')

    vals_x_firststage = model.cbGetNodeRel(x)
    #theta = [var for var in model.getVars() if 'theta' in var.varName]
    #this causes theta[0] to refer to the variable
    vals_theta = model.cbGetNodeRel(theta)
    UB = 0
    LB = 0
    
    for xi in range(Scenarios):

        FirstStageSets = set(SetsIndexedbyScenario[xi].keys()).intersection(BendersSets)
        SuperSetsinScenario = set(SetsIndexedbyScenario[xi].keys()).intersection(SuperSets)
        #for input into the subproblem
        vals_x = {S: 0 for S in SetsIndexedbyScenario[xi].keys()}
        elementscovered = set()
        for S in vals_x_firststage.keys():
          if S in vals_x.keys():
            vals_x[S] = vals_x_firststage[S]
        


                  #add the resulting cut
        SetsContainingBlues = {}
        for b in BlueScenarios[xi].keys():
          SetsContainingBlues[b] = set()
          for S in FirstStageSets:
            if b in SetsIndexedbyScenario[xi][S]:
              SetsContainingBlues[b].add(S)

        #calculate the covers for the super sets
        for b in BlueScenarios[xi].keys():
            vals_x[SuperLookup[(b, xi)]] = max(1 - sum(vals_x[S] for S in SetsContainingBlues[b]), 0)
        # build the subproblem (it's better to build it once, outside the callback, and then modify it inside - left as an exercise)
        vals_y, vals_z, Pi, BDD1ObjVal = subproblemBDDP1(xi,SetsIndexedbyScenario, 
                                                        RedScenarios, BlueScenarios,
                                                        WeightsIndexedbyScenario, 
                                                        vals_x,  output = False)

        if PrintInfo:
          print("\n\n\n")
          print("At iteration " + str(counter) + ", ")
          print("Theta val = ", vals_theta[xi])
          print("Master objective value (LB) = ", vals_theta[xi])
          print("Benders cut bound = ",BDD1ObjVal)
      
        status = BDD1ObjVal - vals_theta[xi] >= 0.001
        if PrintInfo:
          print("Is their any violation?", status)
        if status:
          if PrintInfo: 
            print('Benders cut added...')
            print([vals_z[S]*Pi[S] for S in x.keys()])
            print(vals_z)
            print(vals_x)
            print(Pi)
          # $$\theta \geq 
          # \sum_{r \in \bar{R}(\xi)} c_r \bar{y}_r + 
          # \sum_{S \in \mathcal{S}} \bar{\lambda}_S (x_S - \bar{z}_S) +
          # \sum_{b \in \bigcup_{b\in B(\xi)}}\bar{\lambda}_{S_b} (x^*_{S_b}-\bar{z}_{S_b}) -
          # \sum_{b \in \bigcup_{b\in B(\xi)}} \sum_{S \ni b \subseteq \mathcal{S}}\bar{\lambda}_{S_b} x^*_{S_b} x_S
          # $$ (8)

          BlueElementSetPairs = {(Sb, FirstStageS) for Sb in SuperSetsinScenario for FirstStageS in SetsContainingBlues[bLookup[(Sb, xi)]]}
       
          model.cbLazy(theta[xi] >= sum(vals_y[r]*WeightsIndexedbyScenario[xi][r] for r in vals_y.keys())
                        + gp.quicksum(x[S]*Pi[S] for S in FirstStageSets)
                        - sum(vals_z[S]*Pi[S] for S in FirstStageSets)
                        + sum(Pi[Sb]*(vals_x[Sb] - vals_z[Sb]) for Sb in SuperSetsinScenario)
                        - gp.quicksum(Pi[Sb]*vals_x[Sb]*x[S] for (Sb, S) in BlueElementSetPairs))
          # else:
          #   model.cbLazy(0 >= vals_v.sum()
          #                 + gp.quicksum(x[S]*Pi[S] for S in FirstStageSets)
          #                 - sum(vals_z[S]*Pi[S] for S in x.keys())
          #                 + sum(Pi[Sb]*(vals_x[Sb] - vals_z[Sb]) for Sb in SuperSetsinScenario)
          #                 - gp.quicksum(Pi[Sb]*vals_x[Sb]*x[S] for (Sb, S) in BlueElementSetPairs))
        else:
          if PrintInfo:
            print('No violation')
        #update bounds
        UB += BDD1ObjVal
        LB += vals_theta[xi]
        if PrintInfo:
          print("UB - LB = ", UB - LB)
          print("\n\n\n")
      
        #if we are at the root node and have added a cut
        if model.cbGet(GRB.Callback.MIPSOL_NODCNT) < 1 and IntegerX and status:
          #begin phase #2 by solving the inner lagragian

          #first change the variable types for the fractional master problem solutions
          #force them to be integer, let all the already integer variables 
          #be relaxed and free
          J = {S for S in vals_x.keys() if vals_x[S] 
                <= math.ceil(vals_x[S]) - epsilon and
                vals_x[S] >= math.floor(vals_x[S]) + epsilon}

          #solve the inner lagrange problem 
          vals_y_lagrang, vals_z_lagrang, LagrangianObjVal = subproblemBDDP2(xi, 
                                                                             SetsIndexedbyScenario, 
                                                                             RedScenarios, 
                                                                             BlueScenarios, 
                                                                             WeightsIndexedbyScenario, 
                                                                             vals_x, 
                                                                             Pi, J)
          #phase 2 solve 2.6 to obtain a cut, then go to phase 3

          if PrintInfo:
            print('Lagrangian cut added...')
          #add the resulting cut
          model.cbLazy(theta[xi] >= sum(vals_y_lagrang[r]*WeightsIndexedbyScenario[xi][r] for r in vals_y_lagrang.keys())
              + gp.quicksum(x[S]*Pi[S] for S in FirstStageSets)
              - sum(vals_z_lagrang[S]*Pi[S] for S in FirstStageSets)
              + sum(Pi[Sb]*(vals_x[Sb] - vals_z_lagrang[Sb]) for Sb in SuperSetsinScenario)
              - gp.quicksum(Pi[Sb]*vals_x[Sb]*x[S] for (Sb, S) in BlueElementSetPairs))
          
           #project the cut upwards using the heuristic program
          #initialize parameters for loop
          t = 0 
          change = 1
          lambd_last = Pi
          y_last = vals_y_lagrang
          z_last = vals_z_lagrang
          old_eta = LagrangianObjVal
          delta_t = delta/(t+1)

          #set up the lifting problem
          m_lifting.remove(m_lifting._lambd)
          lambd = m_lifting.addVars(SetsIndexedbyScenario[xi].keys(), name='lambda', vtype = GRB.CONTINUOUS)
          m_lifting._lambd = lambd
         
          #white eta is changing, update the lifting program and then 
          #solve it, then add the resulting strengthed cut to the master
          while t <= IterLim and change > epsilon:
            if PrintInfo:
              print("Lifting Iteration: ", t)
            m_lifting.setObjective(eta  - 
                                   (delta_t/2)*gp.quicksum(
                                       (lambd_last[S] - lambd[S])*(lambd_last[S] - lambd[S])
                                       for S in lambd.keys() ), GRB.MAXIMIZE)
            m_lifting.addConstr(
                eta <= sum(y_last[r]*WeightsIndexedbyScenario[xi][r] for r in y_last.keys()) + 
                gp.quicksum((vals_x[S] - z_last[S])*lambd[S] for S in lambd.keys())
                )
            
            m_lifting.optimize()
            delta_t = delta/(t+1)
            lambd_last = m_lifting.getAttr('x', lambd)
            new_eta = eta.x
            change = abs(new_eta - old_eta)
            old_eta = new_eta
            #resolve inner problem with new lambda 
            vals_y_lagrang, vals_z_lagrang, LagrangianObjVal = subproblemBDDP2(xi, 
                                                                              SetsIndexedbyScenario, 
                                                                              RedScenarios, 
                                                                              BlueScenarios, 
                                                                              WeightsIndexedbyScenario, 
                                                                              vals_x, 
                                                                              lambd_last, J)

            #add the resulting cut

            model.cbLazy(theta[xi] >= sum(vals_y_lagrang[r]*WeightsIndexedbyScenario[xi][r] for r in vals_y_lagrang.keys())
              + gp.quicksum(x[S]*lambd_last[S] for S in FirstStageSets)
              - sum(vals_z_lagrang[S]*lambd_last[S] for S in FirstStageSets)
              + sum(lambd_last[Sb]*(vals_x[Sb] - vals_z_lagrang[Sb]) for Sb in SuperSetsinScenario)
              - gp.quicksum(lambd_last[Sb]*vals_x[Sb]*x[S] for (Sb, S) in BlueElementSetPairs))

            if PrintInfo:
              print('Lifted Lagrangian cut added...')
              print('eta ', new_eta)
              print('change in eta ', change)
              print('**************************************************')
            #update 
            y_last = vals_y_lagrang
            z_last = vals_z_lagrang
            t = t+1

          m_lifting.remove(m_lifting.getConstrs())
        counter += 1
    

### Experiments

In [ ]:
Results = {}
n_elem = 50 #number of elements 
maxBlueProb = 0.2 #
TransmissionProb = 0.9
mipgaptol = 0.05
global LIMIT #time limit
LIMIT = 2*60*60
CoverFactor = 1
N_nodes=5
output = False
random.seed(10)

for N, Scenarios in product([10,15], [15625]):
#for N, Scenarios in product([10, 40], [5]):

  lambd = (n_elem)/np.log(1+N) #average elements in set?

  Sets = {}

  #random scenario generation
  #define each element to be a part of a 
  #graph. With edge probability p 
  #create instance 
  (Sets, ExtensiveReds, ExtensiveBlues, ScenarioFormSets, 
  ExtensiveRedsWeights, SetsIndexedbyScenario, RedScenarios, 
  BlueScenarios, WeightsIndexedbyScenario, SuperLookup, 
  bLookup, BlueMapping) = defineinstance(N,n_elem, Scenarios,
                                          maxBlueProb, TransmissionProb, 
                                          CoverFactor, N_nodes, lambd, Plot = False)

  #extensive form 
  (SelectedReds, SelectedSets, 
  SolnEdges, ObjVal, ObjBound, RunTime) = DeterministicRedBlue(ExtensiveReds, ExtensiveBlues, 
                                                                ScenarioFormSets, ExtensiveRedsWeights, 
                                                                LIMIT, output = False, testing = True, mipgap = mipgaptol)
  Results[('Extensive', n_elem,N,Scenarios)] = [ObjVal, ObjBound, RunTime, True]
  #appx algorithm
  (BestCover, MinWgt, RunTime, success) = test_peleg(Sets, ExtensiveReds, 
                                                    ExtensiveBlues, ScenarioFormSets, 
                                                    ExtensiveRedsWeights)
  Results[('Peleg', n_elem,N,Scenarios)] = [MinWgt, 0 , RunTime, success]
  #Benders and Dual Benders 
  #globals for benders and dual
  counter = 0
  PrintInfo = False
  xWarm = {S:0 for S in ScenarioFormSets.keys()}
  for S in BestCover:
    xWarm[S] = 1
  BendersSets = set(Sets.keys()) #first stage sets not including the super sets 
  SuperSets = set(ScenarioFormSets) - set(BendersSets)
 
  ################################################
  #Standard Benders 
  ################################################

  #Bender Set keys is the first stage sets
  m = gp.Model()
  m.Params.MIPGap = mipgaptol
  m.Params.TimeLimit = LIMIT
  # Create variables
  if not output:
    m.Params.OutputFlag = 0
  x = m.addVars(BendersSets, vtype = GRB.BINARY, lb = 0, ub = 1, name = 'x')
  theta = m.addVars(Scenarios, obj = 1, name='theta', lb = 0)

  #subproblem
  for xi in range(Scenarios):
    #full solution including super sets is in vals_x
    vals_x = {S: 0 for S in SetsIndexedbyScenario[xi].keys()}
    elementscovered = set()
    for S in BestCover.keys():
      if S in vals_x.keys():
        vals_x[S] = 1
        elementscovered = elementscovered.union(set(SetsIndexedbyScenario[xi][S]))
    #get uncovered blue elements 
    uncoveredblues = set(BlueScenarios[xi].keys()) - elementscovered
    if uncoveredblues!= set():
      for b in uncoveredblues:
        vals_x[b] = 1
        
    vals_pi, ObjVal = subproblem(xi,SetsIndexedbyScenario, RedScenarios, WeightsIndexedbyScenario, vals_x)
    FirstStageSets = set(SetsIndexedbyScenario[xi].keys()).intersection(BendersSets)
    SuperSetsinScenario = set(SetsIndexedbyScenario[xi].keys()).intersection(SuperSets)

    RedSetCombos_1 = {(r,S) for S in FirstStageSets 
                      for r in set(SetsIndexedbyScenario[xi][S]).intersection(set(RedScenarios[xi].keys()))}

    RedSetCombos_2 =  {(r,S) for S in SuperSetsinScenario 
                      for r in set(SetsIndexedbyScenario[xi][S]).intersection(set(RedScenarios[xi].keys()))}
    #sets that contain the element associated with the super set 
    SetsContainingBlues = {}
    for b in BlueScenarios[xi].keys():
      SetsContainingBlues[b] = set()
      for S in FirstStageSets:
        if b in SetsIndexedbyScenario[xi][S]:
          SetsContainingBlues[b].add(S)
    Triples = {(r,Sb,FirstStageS) for (r,Sb) in RedSetCombos_2 for FirstStageS in SetsContainingBlues[bLookup[(Sb, xi)]]}
    m.addConstr(theta[xi] >= 
                  sum(x[S]*vals_pi[(r,S)] for (r,S) in RedSetCombos_1)
                  +sum(1*vals_pi[(r,S)]*vals_x[S] for (r,S) in RedSetCombos_2)
                  - sum(x[FirstStageS]*vals_pi[(r,Sb)]*vals_x[Sb]  for (r,Sb,FirstStageS) in Triples))

  # Optimize model
  m._x = x

  m.Params.lazyConstraints = 1 # # To be used with lazy constraints

  m.optimize(benders)
  #vals = m.getAttr('x', x)
  if output:
    print('')
    #print('Optimal x: %s' % str(vals))
    print('Optimal cost: %g' % m.objVal)
    print('')
  Results[('Benders', n_elem,N,Scenarios)] = [m.ObjVal, m.ObjBound , m.RunTime, True]
  ################################################
  #BDD 
  ################################################
  #Params for BDD
  IntegerX = True
  delta = 1
  epsilon  = 0.01
  IterLim = 10
  # for S in BestAugmentedCover:
  #   xWarm[S] = 1

  #heuristic lifting sub problem
  m_lifting = gp.Model()
  #red element set combinatations where r is in S 
  m_lifting.Params.OutputFlag = 0
  eta = m_lifting.addVar(name='eta', lb = 0, vtype = GRB.CONTINUOUS)
  lambd = m_lifting.addVars(SetsIndexedbyScenario[xi].keys(), name='lambda', vtype = GRB.CONTINUOUS)
  m_lifting._lambd = lambd #for easy removal in callback
  # m_lifting.remove(m_lifting._lambd)

  #Bender Set keys is the first stage sets
  m = gp.Model()
  m.Params.MIPGap = mipgaptol
  m.Params.TimeLimit = LIMIT
  if not output:
    m.Params.OutputFlag = 0
  # Create variables
  x = m.addVars(BendersSets, vtype = GRB.BINARY, lb = 0, ub = 1, name = 'x')
  theta = m.addVars(Scenarios, obj = 1, name='theta', lb = 0)


  for xi in range(Scenarios):
    #full solution including super sets is in vals_x
    vals_x = {S: 0 for S in SetsIndexedbyScenario[xi].keys()}
    elementscovered = set()
    for S in BestCover.keys():
      if S in vals_x.keys():
        vals_x[S] = 1
        elementscovered = elementscovered.union(set(SetsIndexedbyScenario[xi][S]))
    #get uncovered blue elements 
    uncoveredblues = set(BlueScenarios[xi].keys()) - elementscovered
    if uncoveredblues!= set():
      for b in uncoveredblues:
        vals_x[b] = 1
        
    vals_pi, ObjVal = subproblem(xi,SetsIndexedbyScenario, RedScenarios, WeightsIndexedbyScenario, vals_x)
    FirstStageSets = set(SetsIndexedbyScenario[xi].keys()).intersection(BendersSets)
    SuperSetsinScenario = set(SetsIndexedbyScenario[xi].keys()).intersection(SuperSets)

    RedSetCombos_1 = {(r,S) for S in FirstStageSets 
                      for r in set(SetsIndexedbyScenario[xi][S]).intersection(set(RedScenarios[xi].keys()))}

    RedSetCombos_2 =  {(r,S) for S in SuperSetsinScenario 
                      for r in set(SetsIndexedbyScenario[xi][S]).intersection(set(RedScenarios[xi].keys()))}
    #sets that contain the element associated with the super set 
    SetsContainingBlues = {}
    for b in BlueScenarios[xi].keys():
      SetsContainingBlues[b] = set()
      for S in FirstStageSets:
        if b in SetsIndexedbyScenario[xi][S]:
          SetsContainingBlues[b].add(S)
    Triples = {(r,Sb,FirstStageS) for (r,Sb) in RedSetCombos_2 for FirstStageS in SetsContainingBlues[bLookup[(Sb, xi)]]}
    m.addConstr(theta[xi] >= 
                  sum(x[S]*vals_pi[(r,S)] for (r,S) in RedSetCombos_1)
                  +sum(1*vals_pi[(r,S)]*vals_x[S] for (r,S) in RedSetCombos_2)
                  - sum(x[FirstStageS]*vals_pi[(r,Sb)]*vals_x[Sb]  for (r,Sb,FirstStageS) in Triples))

  #subproblem

  # Optimize model
  m._x = x
  # for S in x.keys():
  #   x[S].start = xWarm[S]
  # theta.start = sum(WeightsIndexedbyScenario[xi][r] for xi in range(Scenarios) for r in RedScenarios[xi])
  m.Params.lazyConstraints = 1 # # To be used with lazy constraints
  
  m.optimize(BDD)

  #vals = m.getAttr('x', x)

  if output:
    print('')
    #print('Optimal x: %s' % str(vals))
    print('Optimal cost: %g' % m.objVal)
    print('')
  Results[('Dual Benders', n_elem,N,Scenarios)] = [m.ObjVal, m.ObjBound , m.RunTime, True]
  print("Completed instance defined by Nelem, N, Scenarios", (n_elem,N,Scenarios))
  with open("gdrive/My Drive/Colab Notebooks/RBSC/data/computational", 'wb') as fp:
    pkl.dump(Results, fp);

Changed value of parameter MIPGap to 0.05
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 7200.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter MIPGap to 0.05
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 7200.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter MIPGap to 0.05
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 7200.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Completed instance defined by Nelem, N, Scenarios (50, 10, 1500)
Changed value of parameter MIPGap to 0.05
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 7200.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter MIPGap to 0.05
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 7200.0
   Prev: inf  Min: 0.0  Max: inf  Def

### Computer Info

In [ ]:
pd.DataFrame(Results).transpose()

,,,,0,1,2,3
Extensive,50,10,25,37.16,35.36,1.568928,True
Peleg,50,10,25,260.228212,0,4.284551,True
Benders,50,10,25,37.12,35.335838,30.893447,True
Dual Benders,50,10,25,37.12,36.0,27.727044,True
Extensive,50,10,625,36.4016,35.338663,238.8556,True
Peleg,50,10,625,283.215038,0,1635.228347,True
Benders,50,10,625,36.4032,35.271544,636.003433,True
Dual Benders,50,10,625,36.4016,35.044641,515.462988,True
Extensive,50,10,1250,34.0392,32.642577,977.894245,True
Peleg,50,10,1250,0,0,3603.895567,False


In [ ]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         226G   40G  187G  18% /
tmpfs            64M     0   64M   0% /dev
shm             5.8G     0  5.8G   0% /dev/shm
/dev/root       2.0G  1.2G  817M  59% /sbin/docker-init
tmpfs           6.4G  736K  6.4G   1% /var/colab
/dev/sda1       233G   43G  190G  19% /etc/hosts
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive            15G  4.9G   11G  33% /content/gdrive


In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

In [ ]:
!cat /proc/meminfo

MemTotal:       13302912 kB
MemFree:         5360016 kB
MemAvailable:    6461632 kB
Buffers:           18116 kB
Cached:           148408 kB
SwapCached:            0 kB
Active:          6984420 kB
Inactive:         795280 kB
Active(anon):    6457840 kB
Inactive(anon):      864 kB
Active(file):     526580 kB
Inactive(file):   794416 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              1492 kB
Writeback:             0 kB
AnonPages:       7613176 kB
Mapped:            91484 kB
Shmem:              1884 kB
KReclaimable:      44400 kB
Slab:              89968 kB
SReclaimable:      44400 kB
SUnreclaim:        45568 kB
KernelStack:        5744 kB
PageTables:        23808 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6651456 kB
Committed_AS:   14355768 kB
VmallocTotal:   34359738367 kB
VmallocUsed:        8228 kB
VmallocChunk:          0 kB
Percpu:          